In [1]:
from pathlib import Path
from bs4 import BeautifulSoup
import datetime
import re
import json
import time
import pandas as pd
import json

import bokeh
from bokeh.plotting import figure, show, save, output_file, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource,LinearColorMapper,BasicTicker, PrintfTickFormatter, ColorBar
import datetime
from bokeh.models import NumeralTickFormatter
from bokeh.models import Span
from bokeh.models import Legend, LegendItem, Label, LabelSet

In [2]:
def commonFixes(json_str: str):
    json_str = re.sub(r'quot;', '', json_str)
    json_str = re.sub(r'&qu', '"', json_str)
    return json_str

# change the html file name here
page = Path(r'./data/SouthKorea-cycling.html')
soup = BeautifulSoup(page.open(), 'html.parser')
data_elems = soup.find_all('div', {'data-react-class': "View"})
possible_data = data_elems[0].get_attribute_list('data-react-props')[0]
possible_data = commonFixes(possible_data)
d = json.loads(possible_data)  # if they load they are clean ;O
json.dump(d, Path(f'./data/{page.name}_{int(time.time())}.json').open('w'))

print('end')

end


In [3]:
with open('./data/Netherland-cycling.html_1588694478.json') as f:
    dutchcycling = json.load(f)
#print (dutchcycling)

with open('./data/Netherland-Running.html_1588695022.json') as f:
    dutchrunning = json.load(f)


with open('./data/Brazil-cycling.html_1588696155.json') as f:
    brazilcycling = json.load(f)


with open('./data/Brazil-running.html_1588696161.json') as f:
    brazilrunning = json.load(f)
    
with open('./data/Mexico-cycling.html_1588696170.json') as f:
    mexicocycling = json.load(f)


with open('./data/Mexico-running.html_1588696168.json') as f:
    mexicorunning = json.load(f)
    
with open('./data/SouthAfrica-cycling.html_1588696182.json') as f:
    southafricacycling = json.load(f)


with open('./data/SouthAfrica-running.html_1588696185.json') as f:
    southafricarunning = json.load(f)
    
with open('./data/SouthKorea-cycling.html_1588696194.json') as f:
    southkoreacycling = json.load(f)


with open('./data/SouthKorea-running.html_1588696192.json') as f:
    southkorearunning = json.load(f)
    
with open('./data/Glasgow-bike.html_1589378615.json') as f:
    glasgowcycling = json.load(f)
    
with open('./data/Glasgow-run.html_1589378625.json') as f:
    glasgowrunning = json.load(f)
    
with open('./data/India-bike.html_1589931744.json') as f:
    indiacycling = json.load(f)
    
with open('./data/India-run.html_1589931739.json') as f:
    indiarunning = json.load(f)
    
with open('./data/Jakarta-bike.html_1589931768.json') as f:
    jakartacycling = json.load(f)

with open('./data/Jakarta-run.html_1589931821.json') as f:
    jakartarunning = json.load(f)
    
with open('./data/Scotland-run.html_1589378629.json') as f:
    scotlandrunning = json.load(f)
    
with open('./data/Scotland-bike.html_1589378631.json') as f:
    scotlandcycling = json.load(f)

with open('./data/HK-bike.html_1589931725.json') as f:
    hkcycling = json.load(f)
#print (hkcycling)
   
with open('./data/HK-run.html_1589931729.json') as f:
    hkrunning = json.load(f)

In [4]:
#year by visualisation in countries with lockdown
def visualise_country_lockdown(json_data, lockdown_str, figure_title):
    data = pd.DataFrame(json_data['weeklyStats'])
    data['year'] = data.date_part.apply(lambda date: date[:4])
    data['date'] = data.date_part.apply(lambda date: date[5:])
    data['week'] = data.date_part.apply(lambda date: datetime.datetime.strptime(date, '%Y-%m-%d').isocalendar()[1])
    data2018 = data[data.year=="2018"].sort_values('week')
    data2019  = data[data.year=="2019"].sort_values('week')
    data2020  = data[data.year=="2020"].sort_values('week')
    
    
    TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom'
    p = figure(title=figure_title, y_axis_type="linear",x_axis_type='linear', tools = TOOLS, plot_width=700, plot_height=400)


    p.line(data2018['week'], data2018.total_activities, legend_label="2018", line_color="purple", line_width = 2, alpha=0.5)
    p.line(data2019['week'], data2019.total_activities, legend_label="2019", line_color="blue", line_width = 2, alpha=0.5)
    p.line(data2020['week'], data2020.total_activities, legend_label="2020", line_color = 'green', line_width = 2, alpha=0.5)

    p.xgrid[0].grid_line_color=None
    p.ygrid[0].grid_line_alpha=0.5

    #legend = Legend(items=[LegendItem(label=lockdown_start_week, renderers=[vline], index=0)])
    if lockdown_str is not None:
        lockdown_start_week = datetime.datetime.strptime(lockdown_str, '%Y-%m-%d').isocalendar()[1]
        vline = Span(location=lockdown_start_week, dimension='height', line_color='red', line_width=1.5)
        p.renderers.extend([vline])

        lockdown_label = Label(x=170, y=310, x_units='screen', y_units='screen',
                          text='Start of lockdown: {}'.format(lockdown_str), render_mode='css',
                          border_line_color='black', border_line_alpha=0,
                          background_fill_color='white',text_color='red', text_font_size='10pt' ,background_fill_alpha=1.0)     
        p.add_layout(lockdown_label)

    p.legend.location = "top_left"

    p.xaxis.axis_label = 'Week'
    p.yaxis.axis_label = 'Count'



    output_file(f"plots_age/{figure_title}.html", title=figure_title)
    save(p)



In [5]:
countries_list = [
    [dutchrunning, '2020-03-15', 'Amsterdam Weekly Running'],
    [dutchcycling, '2020-03-15', 'Amsterdam Weekly Cycling'], 
    [mexicorunning, '2020-03-22', 'Mexico City Weekly Running'],
    [mexicocycling, '2020-03-22', 'Mexico City Weekly Cycling'],
    [brazilcycling, '2020-03-24', 'Sao Paulo Weekly Cycling'],
    [brazilrunning, '2020-03-24', 'Sao Paulo Weekly Running'],
    [southafricarunning, '2020-03-26', 'Johannesburg Weekly Running'],
    [southafricacycling, '2020-03-26', 'Johannesburg Weekly Cycling'],
    [southkorearunning, None, 'Seoul Weekly Running'],
    [southkoreacycling, None, 'Seoul Weekly Cycling'], 
    [indiarunning, '2020-03-25', 'Delhi Weekly Running'],
    [indiacycling, '2020-03-25', 'Delhi Weekly Cycling'],
    [jakartarunning, None, 'Jakarta Weekly Running'],
    [jakartacycling, None, 'Jakarta Weekly Cycling'],
    [glasgowrunning, '2020-03-23', 'Glasgow Weekly Running'],
    [glasgowcycling, '2020-03-23', 'Glasgow Weekly Cycling'],
    [scotlandrunning, '2020-03-23', 'Scotland Weekly Running'],
    [scotlandcycling, '2020-03-23', 'Scotland Weekly Cycling'], 
    [hkrunning, None, 'Hong Kong Weekly Running'],
    [hkcycling, None, 'Hong Kong Weekly Cycling']
]

In [ ]:
for country in countries_list:
    visualise_country_lockdown(*country)

In [8]:
def visualise_country_age(json_data, figure_title):
    data = pd.DataFrame(json_data['weeklyStats'])
    data['year'] = data.date_part.apply(lambda date: date[:4])
    table_columns=['year', 'under_20_athletes', 'under_35_athletes', 'under_55_athletes', 'under_65_athletes', 'over_65_athletes']
    table=data.loc[:, table_columns]
    table=table.groupby(['year']).sum().reset_index()
    table.index.name = 'categories'
    
    category_names=['Under 20', 'Under 35', 'Under 55', 'Under 65', 'Over 65']
    categories = table_columns[1:]
    
    TOOLS = "save,pan,box_zoom,reset,wheel_zoom,tap"

    source = ColumnDataSource(data=table)
    p = figure( plot_width=800, title=figure_title ,toolbar_location='above', tools=TOOLS)
    colors = brewer['Dark2'][5]
    p.vbar_stack(categories, x='categories', width=0.9, source=source, color=colors,
                 legend_label=category_names)

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    xticks = [0, 1, 2]
    p.xaxis.ticker = xticks
    p.xaxis.major_label_overrides = {tick:value for tick, value in zip(xticks, list(table.year))}
    p.yaxis.formatter=NumeralTickFormatter(format="00")
    p.xgrid.grid_line_color = None
    p.axis.minor_tick_line_color = None
    p.outline_line_color = None
    p.xaxis.axis_label = 'Year'
    p.yaxis.axis_label = 'Total Cyclists'
    p.legend.location = "top_right"
    p.legend.orientation = "vertical"


    output_file("stacked_bar.html", title=figure_title)

    show(p)

In [9]:
visualise_country_age(hkrunning, "Hong Kong Weekly Running")

In [6]:
#age group visualisation with lockdown
def visualisation_country_age_lockdown(json_data, lockdown_str, figure_title):
    data = pd.DataFrame(json_data['weeklyStats'])
    data['year'] = data.date_part.apply(lambda date: date[:4])
    data['week'] = data.date_part.apply(lambda date: datetime.datetime.strptime(date, '%Y-%m-%d').isocalendar()[1])
    data2020  = data[data.year=="2020"].sort_values('week')


    TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom'
    p = figure(title=figure_title, y_axis_type="linear",x_axis_type='linear', tools = TOOLS, plot_width=700, plot_height=400)


    p.line(data2020['week'], data2020.under_20_athletes, legend_label="Under 20", line_color="purple", line_width = 2, alpha=0.5)
    p.line(data2020['week'], data2020.under_35_athletes, legend_label="20-34", line_color="blue", line_width = 2, alpha=0.5)
    p.line(data2020['week'], data2020.under_55_athletes, legend_label="35-54", line_color = 'green', line_width = 2, alpha=0.5)
    p.line(data2020['week'], data2020.under_65_athletes, legend_label="55-64", line_color="red", line_width = 2, alpha=0.5)
    p.line(data2020['week'], data2020.over_65_athletes, legend_label="Over 65", line_color="orange", line_width = 2, alpha=0.5)

    p.xgrid[0].grid_line_color=None
    p.ygrid[0].grid_line_alpha=0.5

    if lockdown_str is not None:
        lockdown_start_week = datetime.datetime.strptime(lockdown_str, '%Y-%m-%d').isocalendar()[1]

        vline = Span(location=lockdown_start_week, dimension='height', line_color='red', line_width=1.5)
        p.renderers.extend([vline])
   
        lockdown_label = Label(x=430, y=310, x_units='screen', y_units='screen',
                         text='Start of lockdown: {}'.format(lockdown_str), render_mode='css',
                         border_line_color='black', border_line_alpha=0,
                         background_fill_color='white',text_color='red', text_font_size='10pt' ,background_fill_alpha=1.0)
        p.add_layout(lockdown_label)

    p.legend.location = "top_left"

    p.xaxis.axis_label = 'Week'
    p.yaxis.axis_label = 'Count'


   
    output_file(f"plots_age/{figure_title}.html", title=figure_title)
    save(p)

In [10]:
countries_list_age = [
    [dutchrunning, '2020-03-15', 'Amsterdam Running by Age'],
    [dutchcycling, '2020-03-15', 'Amsterdam Cycling by Age'], 
    [mexicorunning, '2020-03-22', 'Mexico City Running by Age'],
    [mexicocycling, '2020-03-22', 'Mexico City Cycling by Age'],
    [brazilcycling, '2020-03-24', 'Sao Paulo Cycling by Age'],
    [brazilrunning, '2020-03-24', 'Sao Paulo Running by Age'],
    [southafricarunning, '2020-03-26', 'Johannesburg Running by Age'],
    [southafricacycling, '2020-03-26', 'Johannesburg Cycling by Age'],
    [southkorearunning, None, 'Seoul Running by Age'],
    [southkoreacycling, None, 'Seoul Cycling by Age'], 
    [indiarunning, '2020-03-25', 'Delhi Running by Age'],
    [indiacycling, '2020-03-25', 'Delhi Cycling by Age'],
    [jakartarunning, None, 'Jakarta Running by Age'],
    [jakartacycling, None, 'Jakarta Cycling by Age'],
    [glasgowrunning, '2020-03-23', 'Glasgow Running by Age'],
    [glasgowcycling, '2020-03-23', 'Glasgow Cycling by Age'],
    [scotlandrunning, '2020-03-23', 'Scotland Running by Age'],
    [scotlandcycling, '2020-03-23', 'Scotland Cycling by Age'], 
    [hkrunning, None, 'Hong Kong Running by Age'],
    [hkcycling, None, 'Hong Kong Cycling by Age']
]

In [ ]:
for country in countries_list_age:
    visualisation_country_age_lockdown(*country)

In [ ]:
#age group visualisation without lockdown
def visualisation_country_age(json_data, figure_title):
    data = pd.DataFrame(json_data['weeklyStats'])
    data['year'] = data.date_part.apply(lambda date: date[:4])
    data['week'] = data.date_part.apply(lambda date: datetime.datetime.strptime(date, '%Y-%m-%d').isocalendar()[1])
    data2020  = data[data.year=="2020"].sort_values('week')
    #lockdown_start_week = datetime.datetime.strptime(lockdown_str, '%Y-%m-%d').isocalendar()[1]


    TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom'
    p = figure(title=figure_title, y_axis_type="linear",x_axis_type='linear', tools = TOOLS, plot_width=700, plot_height=400)


    p.line(data2020['week'], data2020.under_20_athletes, legend="Under 20", line_color="purple", line_width = 2, alpha=0.5)
    p.line(data2020['week'], data2020.under_35_athletes, legend="20-34", line_color="blue", line_width = 2, alpha=0.5)
    p.line(data2020['week'], data2020.under_55_athletes, legend="35-54", line_color = 'green', line_width = 2, alpha=0.5)
    p.line(data2020['week'], data2020.under_65_athletes, legend="55-64", line_color="red", line_width = 2, alpha=0.5)
    p.line(data2020['week'], data2020.over_65_athletes, legend="Over 65", line_color="orange", line_width = 2, alpha=0.5)

   # vline = Span(location=lockdown_start_week, dimension='height', line_color='red', line_width=1.5)
    #p.renderers.extend([vline])

    p.xgrid[0].grid_line_color=None
    p.ygrid[0].grid_line_alpha=0.5

   
    #lockdown_label = Label(x=430, y=310, x_units='screen', y_units='screen',
    #                 text='Start of lockdown', render_mode='css',
    #                 border_line_color='black', border_line_alpha=0,
    #                 background_fill_color='white',text_color='red', text_font_size='10pt' ,background_fill_alpha=1.0)
    #p.add_layout(lockdown_label)

    p.legend.location = "top_left"

    p.xaxis.axis_label = 'Week'
    p.yaxis.axis_label = 'Count'



    output_file("DutchWeeklyRunning_age.html", title=figure_title)
    show(p)

In [ ]:
visualisation_country_age(hkcycling, "Hong Kong Cycling by Age" )